In [77]:
# getting started, importing the libraries

import pandas as pd
import matplotlib.pyplot as plt
import pathlib2 as pathlib
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [123]:
#setting the right path to load the data

cwd=pathlib.Path.cwd()
datadirpath=cwd.joinpath("data")
rawdatapath=datadirpath.joinpath("raw")

In [124]:
# load the data about the reviews of individual users, deleting the first index column, printing the left columns
reviewdf=pd.read_csv(rawdatapath.joinpath("collaborative_books_df.csv"))
reviewdf=reviewdf.drop([reviewdf.columns[0]],axis=1) # removed unnamed index column
print(reviewdf.columns)

Index(['title', 'book_id', 'user_id_mapping', 'book_id_mapping',
       'Predicted Rating', 'Actual Rating'],
      dtype='object')


In [125]:
#looking at the dimensions of the dataset

print (reviewdf.shape)

(196296, 6)


In [127]:
#loading the metadata of the 96 books that we have

metadatadf=pd.read_csv(rawdatapath.joinpath("collaborative_book_metadata.csv"))
metadatadf=metadatadf.drop([metadatadf.columns[0]],axis=1) # removed unnamed index column
print(metadatadf.columns)


Index(['book_id', 'title', 'image_url', 'url', 'num_pages', 'ratings_count',
       'description', 'genre', 'name', 'book_id_mapping'],
      dtype='object')


In [128]:

# Merge the 2 datasets  on 'book_id' to get enriched data of reviewd books
merged_df = pd.merge(reviewdf, metadatadf, on='book_id')

#  Save the merged DataFrame to a new CSV file (just in case)
merged_df.to_csv('merged_books.csv', index=False)

# Print the first few rows of the merged DataFrame
print(merged_df.shape) # only 17.248 of 196296 lines are left, so we have 17.248 reviews for the 96 books
print(merged_df.head())

(17248, 15)
              title_x  book_id  user_id_mapping  book_id_mapping_x  \
0  I Am the Messenger    19057             1537                299   
1  I Am the Messenger    19057            23039                299   
2  I Am the Messenger    19057            39096                299   
3  I Am the Messenger    19057            14631                299   
4  I Am the Messenger    19057            32816                299   

   Predicted Rating  Actual Rating             title_y  \
0               4.5              5  I Am the Messenger   
1               4.9              3  I Am the Messenger   
2               3.9              3  I Am the Messenger   
3               4.7              4  I Am the Messenger   
4               4.3              5  I Am the Messenger   

                                           image_url  \
0  https://images.gr-assets.com/books/1398483261m...   
1  https://images.gr-assets.com/books/1398483261m...   
2  https://images.gr-assets.com/books/1398483261m.

In [129]:


# ensure, we have no rows, that do not have the additional metadata Find rows with empty (NaN) values in the 'num_pages' column
empty_num_pages_count = merged_df['image_url'].isna().sum()

# Print the result
print(f'There are {empty_num_pages_count} rows with an empty "num_pages" column.')
print (merged_df.shape)
print(merged_df.columns)

There are 0 rows with an empty "num_pages" column.
(17248, 15)
Index(['title_x', 'book_id', 'user_id_mapping', 'book_id_mapping_x',
       'Predicted Rating', 'Actual Rating', 'title_y', 'image_url', 'url',
       'num_pages', 'ratings_count', 'description', 'genre', 'name',
       'book_id_mapping_y'],
      dtype='object')


In [130]:

#Compare the columns 'title_x' and 'title_y' and find out if there is any row where there are differences
differences = merged_df['title_x'] != merged_df['title_y']

# Find out how many rows have different values
num_differences = differences.sum()
print(f'There are {num_differences} rows with different values in "title_x" and "title_y".')

# Display the rows with differences
diff_rows = merged_df[differences]
print(diff_rows[['title_x', 'title_y']])

There are 0 rows with different values in "title_x" and "title_y".
Empty DataFrame
Columns: [title_x, title_y]
Index: []


In [131]:

# Deleting the 'title_y' column as it is similar to 'title_x', see verification above
merged_df = merged_df.drop(merged_df.columns[6],axis=1)
#reviewdf=reviewdf.drop([reviewdf.columns[0]],axis=1) # removed unnamed index column


# To confirm the column is deleted, print the  columns again
print(merged_df.columns)

Index(['title_x', 'book_id', 'user_id_mapping', 'book_id_mapping_x',
       'Predicted Rating', 'Actual Rating', 'image_url', 'url', 'num_pages',
       'ratings_count', 'description', 'genre', 'name', 'book_id_mapping_y'],
      dtype='object')


In [132]:
# Transform data about users and their ratings, so that each user is represented by one row, that includes their rating history
#  create a user-item matrix
user_item_matrix = merged_df.pivot(index='user_id_mapping', 
                                   columns='book_id_mapping_x', 
                                   values='Actual Rating')

In [133]:
#handle mising values - as a next step find out how to import the average rating from "book_titles.csv"
imputer = SimpleImputer(strategy='mean')
user_item_matrix_imputed = pd.DataFrame(imputer.fit_transform(user_item_matrix),
                                        columns=user_item_matrix.columns,
                                        index=user_item_matrix.index)

In [74]:
# erst mal nicht machen, hat nicht geholfen
# Convert all genre names to lowercase and strip whitespace
#merged_df['genre_split'] = merged_df['genre_split'].str.lower().str.strip()

In [134]:

# First, split the 'genre' column into multiple rows
# Explode splits lists of genres into separate rows per genre
# 
merged_df['genre_split'] = merged_df['genre'].str.split(', ')
merged_df = merged_df.explode('genre_split')

# Now apply one-hot encoding to the 'genre_split' column
genre_one_hot = pd.get_dummies(merged_df['genre_split'], prefix='Genre')

# Group by 'user_id_mapping' and 'book_id_mapping_x' and sum the one-hot encoded genres
merged_df = merged_df.drop(columns=['genre'])  # Drop original genre if no longer needed
merged_df = pd.concat([merged_df, genre_one_hot], axis=1)

# You may need to re-aggregate if you had exploded the DataFrame:
merged_df = merged_df.groupby(['user_id_mapping', 'book_id_mapping_x'], as_index=False).sum()

In [75]:
#erst mal nicht, hat nicht geholfen 
# once more one-hot enconding with cleaned up genre columns
# Perform one-hot encoding again on the standardized genres
#genre_one_hot = pd.get_dummies(merged_df['genre_split'], prefix='Genre')

# Concatenate the one-hot encoded genres with the original DataFrame
#merged_df = pd.concat([merged_df, genre_one_hot], axis=1)

In [135]:
print(merged_df.columns)
print(merged_df.head)

Index(['user_id_mapping', 'book_id_mapping_x', 'title_x', 'book_id',
       'Predicted Rating', 'Actual Rating', 'image_url', 'url', 'num_pages',
       'ratings_count', 'description', 'name', 'book_id_mapping_y',
       'genre_split', 'Genre_'children'', 'Genre_'children']', 'Genre_'comics',
       'Genre_'fantasy', 'Genre_'fiction'', 'Genre_'fiction']',
       'Genre_'history', 'Genre_'mystery', 'Genre_'non-fiction'',
       'Genre_'non-fiction']', 'Genre_'poetry'', 'Genre_'poetry']',
       'Genre_'romance'', 'Genre_'romance']', 'Genre_'young-adult'',
       'Genre_'young-adult']', 'Genre_['children'', 'Genre_['comics',
       'Genre_['fantasy', 'Genre_['fiction'', 'Genre_['mystery',
       'Genre_['non-fiction'', 'Genre_['non-fiction']', 'Genre_['poetry'',
       'Genre_['romance'', 'Genre_['young-adult'', 'Genre_biography'',
       'Genre_biography']', 'Genre_crime'', 'Genre_crime']', 'Genre_graphic'',
       'Genre_graphic']', 'Genre_historical fiction', 'Genre_paranormal'',
    

In [136]:
import re

# Function to clean up genre names
def clean_genre_name(genre_name):
    # Remove leading/trailing quotes and brackets
    cleaned_name = re.sub(r"[\[\]']+", "", genre_name)
    # Convert to lowercase and strip leading/trailing whitespace
    cleaned_name = cleaned_name.strip().lower()
    return cleaned_name

# Clean up the column names in the DataFrame
cleaned_columns = {col: clean_genre_name(col) for col in merged_df.columns if col.startswith('Genre_')}
merged_df.rename(columns=cleaned_columns, inplace=True)
print(merged_df.columns)

Index(['user_id_mapping', 'book_id_mapping_x', 'title_x', 'book_id',
       'Predicted Rating', 'Actual Rating', 'image_url', 'url', 'num_pages',
       'ratings_count', 'description', 'name', 'book_id_mapping_y',
       'genre_split', 'genre_children', 'genre_children', 'genre_comics',
       'genre_fantasy', 'genre_fiction', 'genre_fiction', 'genre_history',
       'genre_mystery', 'genre_non-fiction', 'genre_non-fiction',
       'genre_poetry', 'genre_poetry', 'genre_romance', 'genre_romance',
       'genre_young-adult', 'genre_young-adult', 'genre_children',
       'genre_comics', 'genre_fantasy', 'genre_fiction', 'genre_mystery',
       'genre_non-fiction', 'genre_non-fiction', 'genre_poetry',
       'genre_romance', 'genre_young-adult', 'genre_biography',
       'genre_biography', 'genre_crime', 'genre_crime', 'genre_graphic',
       'genre_graphic', 'genre_historical fiction', 'genre_paranormal',
       'genre_paranormal', 'genre_thriller'],
      dtype='object')


In [137]:
# Create a mapping for similar genres to merge them
# This step assumes that after cleaning, some genres will have identical names and should be merged
unique_genres = list(set(cleaned_columns.values()))  # Get unique cleaned genre names

# Initialize columns for cleaned genres if not already existing
for genre in unique_genres:
    if genre not in merged_df.columns:
        merged_df[genre] = 0

# Sum the columns that are considered similar
for original_col in cleaned_columns.values():
    merged_df[original_col] = merged_df.filter(like=original_col).sum(axis=1)

# Remove duplicate columns
merged_df = merged_df.loc[:, ~merged_df.columns.duplicated()]

# Optionally, remove any genres with no entries after merging
merged_df = merged_df.loc[:, (merged_df != 0).any(axis=0)]
print(merged_df.columns)

Index(['user_id_mapping', 'book_id_mapping_x', 'title_x', 'book_id',
       'Predicted Rating', 'Actual Rating', 'image_url', 'url', 'num_pages',
       'ratings_count', 'description', 'name', 'book_id_mapping_y',
       'genre_split', 'genre_children', 'genre_comics', 'genre_fantasy',
       'genre_fiction', 'genre_history', 'genre_mystery', 'genre_non-fiction',
       'genre_poetry', 'genre_romance', 'genre_young-adult', 'genre_biography',
       'genre_crime', 'genre_graphic', 'genre_historical fiction',
       'genre_paranormal', 'genre_thriller'],
      dtype='object')


In [143]:
print(merged_df.head())
print(merged_df.columns)

   user_id_mapping  book_id_mapping_x  \
0                0                413   
1                0                753   
2                0                847   
3                1                268   
4                3                846   

                                             title_x   book_id  \
0  Pride and Prejudice and Zombies Pride and Prej...  64897569   
1  Persepolis The Story of a Childhood Persepolis...     76128   
2  A Child Called It Dave Pelzer 1A Child Called ...    728976   
3  Sharp ObjectsSharp ObjectsSharp ObjectsSharp O...    266236   
4  The Coincidence of Callie Kayden The Coinciden...  48341373   

   Predicted Rating  Actual Rating  \
0              33.0             44   
1              37.6             40   
2              33.6             36   
3              12.8             16   
4              11.4             12   

                                           image_url  \
0  https://images.gr-assets.com/books/1320449653m...   
1  https://imag

In [144]:
# Extract relevant columns from merged_df
book_metadata = merged_df[['book_id_mapping_x', 'num_pages', 'ratings_count'] + genre_columns].drop_duplicates()

# Check the head of book_metadata to ensure it looks correct
print(book_metadata.head())
print(book_metadata.columns)

   book_id_mapping_x  num_pages  ratings_count  \
0                413       3520        1160907   
1                753       1224         955760   
2                847       2208        3771432   
3                268       1016         833576   
4                846        849         295314   

                                         genre_split  genre_children  \
0  ['fantasyparanormal''romance''fiction''history...               0   
1  ['comicsgraphic''non-fiction''historyhistorica...               0   
2  ['non-fiction''historyhistorical fictionbiogra...               9   
3                  ['mysterythrillercrime''fiction']               0   
4                  ['romance''young-adult''fiction']               0   

   genre_comics  genre_fantasy  genre_fiction  genre_history  genre_mystery  \
0             0              2              9              1              2   
1             2              0              9              1              0   
2             2              

In [145]:
print(user_item_matrix_imputed.head())
print(user_item_matrix_imputed.columns)

user_id_mapping       0         1         3         4         7         9      \
book_id_mapping_x                                                               
3                  3.745690  3.745690  3.745690  3.745690  3.745690  3.745690   
38                 4.006250  4.006250  4.006250  4.006250  4.006250  4.006250   
79                 4.210938  4.210938  4.210938  4.210938  4.210938  4.210938   
93                 4.245935  4.245935  4.245935  4.245935  4.245935  4.245935   
99                 3.310160  3.310160  3.310160  3.310160  3.310160  3.310160   

user_id_mapping       11        12        21        33     ...     83482  \
book_id_mapping_x                                          ...             
3                  3.745690  3.745690  3.745690  3.745690  ...  3.745690   
38                 4.006250  4.006250  4.006250  4.006250  ...  4.006250   
79                 4.210938  4.210938  4.210938  4.210938  ...  4.210938   
93                 4.245935  4.245935  4.245935  4.2

In [146]:
# Set 'book_id_mapping_x' as index for merging
book_metadata = book_metadata.set_index('book_id_mapping_x')

# Transpose the user-item matrix to align book IDs as index
user_item_matrix_imputed = user_item_matrix_imputed.T

# Merge metadata into the user-item matrix
user_item_with_metadata = user_item_matrix_imputed.join(book_metadata, how='left')

# Transpose back to the original format
user_item_with_metadata = user_item_with_metadata.T

# Verify columns after merging
print(user_item_with_metadata.head())
print(user_item_with_metadata.columns)

user_id_mapping     0         1         3         4         7         9      \
3                 3.74569   3.74569   3.74569   3.74569   3.74569   3.74569   
38                4.00625   4.00625   4.00625   4.00625   4.00625   4.00625   
79               4.210938  4.210938  4.210938  4.210938  4.210938  4.210938   
93               4.245935  4.245935  4.245935  4.245935  4.245935  4.245935   
99                3.31016   3.31016   3.31016   3.31016   3.31016   3.31016   

user_id_mapping     11        12        21        33     ...     83482  \
3                 3.74569   3.74569   3.74569   3.74569  ...   3.74569   
38                4.00625   4.00625   4.00625   4.00625  ...   4.00625   
79               4.210938  4.210938  4.210938  4.210938  ...  4.210938   
93               4.245935  4.245935  4.245935  4.245935  ...  4.245935   
99                3.31016   3.31016   3.31016   3.31016  ...   3.31016   

user_id_mapping     83532     83533     83695     83815     83971     84075  \
3

In [138]:
#should not be needed anymore
#  Remove 'genre' reference from book_metadata creation
#book_metadata = merged_df[['book_id_mapping_x', 'num_pages', 'ratings_count']].drop_duplicates()

# Add the one-hot encoded genre columns to book_metadata
#genre_columns = [col for col in merged_df.columns if col.startswith('genre_')]
#book_metadata = pd.concat([book_metadata, merged_df[genre_columns]], axis=1).drop_duplicates()
#print(book_metadata.columns)

Index(['book_id_mapping_x', 'num_pages', 'ratings_count', 'genre_split',
       'genre_children', 'genre_comics', 'genre_fantasy', 'genre_fiction',
       'genre_history', 'genre_mystery', 'genre_non-fiction', 'genre_poetry',
       'genre_romance', 'genre_young-adult', 'genre_biography', 'genre_crime',
       'genre_graphic', 'genre_historical fiction', 'genre_paranormal',
       'genre_thriller'],
      dtype='object')


In [119]:
#print(merged_df.columns)

Index(['user_id_mapping', 'book_id_mapping_x', 'title_x', 'book_id',
       'Predicted Rating', 'Actual Rating', 'image_url', 'url', 'num_pages',
       'ratings_count', 'description', 'name', 'book_id_mapping_y',
       'genre_split', 'genre_children', 'genre_comics', 'genre_fantasy',
       'genre_fiction', 'genre_history', 'genre_mystery', 'genre_non-fiction',
       'genre_poetry', 'genre_romance', 'genre_young-adult', 'genre_biography',
       'genre_crime', 'genre_graphic', 'genre_historical fiction',
       'genre_paranormal', 'genre_thriller'],
      dtype='object')


In [142]:
#print(book_metadata.head)

<bound method NDFrame.head of      book_id_mapping_x  num_pages  ratings_count  \
0                413.0     3520.0      1160907.0   
1                753.0     1224.0       955760.0   
2                847.0     2208.0      3771432.0   
3                268.0     1016.0       833576.0   
4                846.0      849.0       295314.0   
..                 ...        ...            ...   
403                NaN        NaN            NaN   
475                NaN        NaN            NaN   
531                NaN        NaN            NaN   
678                NaN        NaN            NaN   
804                NaN        NaN            NaN   

                                           genre_split  genre_children  \
0    ['fantasyparanormal''romance''fiction''history...               0   
1    ['comicsgraphic''non-fiction''historyhistorica...               0   
2    ['non-fiction''historyhistorical fictionbiogra...               9   
3                    ['mysterythrillercrime''fict

In [139]:
# Example DataFrames to check their structure
#print(user_item_matrix_imputed.head())
#print(book_metadata.head())

book_id_mapping_x      3        38        79        93       99        116  \
user_id_mapping                                                              
0                  3.74569  4.00625  4.210938  4.245935  3.31016  4.231293   
1                  3.74569  4.00625  4.210938  4.245935  3.31016  4.231293   
3                  3.74569  4.00625  4.210938  4.245935  3.31016  4.231293   
4                  3.74569  4.00625  4.210938  4.245935  3.31016  4.231293   
7                  3.74569  4.00625  4.210938  4.245935  3.31016  4.231293   

book_id_mapping_x       137       144       158       170  ...       813  \
user_id_mapping                                            ...             
0                  4.205882  4.333333  4.361345  3.888112  ...  4.142857   
1                  4.205882  4.333333  4.361345  3.888112  ...  4.142857   
3                  4.205882  4.333333  4.361345  3.888112  ...  4.142857   
4                  4.205882  4.333333  4.361345  3.888112  ...  4.142857 

In [140]:
# Ensure that user_item_matrix_imputed is indexed by book IDs
#user_item_matrix_imputed = user_item_matrix_imputed.T  # Transpose to get book IDs as index

# Merge metadata with user-item matrix
#user_item_with_metadata = user_item_matrix_imputed.join(
    book_metadata.set_index('book_id_mapping_x'),
    how='left'
)

# Transpose back to the original format
#user_item_with_metadata = user_item_with_metadata.T

In [141]:
#print(user_item_with_metadata.columns)

Index([  3.0,  38.0,  79.0,  93.0,  99.0, 116.0, 137.0, 144.0, 158.0, 170.0,
       186.0, 205.0, 217.0, 220.0, 221.0, 224.0, 230.0, 231.0, 235.0, 248.0,
       260.0, 262.0, 268.0, 271.0, 272.0, 286.0, 293.0, 299.0, 313.0, 315.0,
       318.0, 339.0, 348.0, 350.0, 406.0, 410.0, 411.0, 413.0, 419.0, 421.0,
       436.0, 437.0, 440.0, 450.0, 456.0, 479.0, 481.0, 490.0, 497.0, 505.0,
       513.0, 524.0, 528.0, 530.0, 543.0, 544.0, 553.0, 562.0, 569.0, 571.0,
       578.0, 580.0, 589.0, 594.0, 601.0, 626.0, 627.0, 636.0, 651.0, 674.0,
       680.0, 691.0, 693.0, 697.0, 707.0, 719.0, 753.0, 765.0, 767.0, 769.0,
       779.0, 780.0, 790.0, 802.0, 810.0, 812.0, 813.0, 836.0, 846.0, 847.0,
       855.0, 868.0, 873.0, 878.0, 882.0, 914.0],
      dtype='float64', name='book_id_mapping_x')


In [122]:
# Transpose user_item_matrix_imputed for merging, then transpose back
#user_item_with_metadata = user_item_matrix_imputed.T.join(book_metadata.set_index('book_id_mapping_x')).T
#print(user_item_with_metadata.columns)

Index([  3.0,  38.0,  79.0,  93.0,  99.0, 116.0, 137.0, 144.0, 158.0, 170.0,
       186.0, 205.0, 217.0, 220.0, 221.0, 224.0, 230.0, 231.0, 235.0, 248.0,
       260.0, 262.0, 268.0, 271.0, 272.0, 286.0, 293.0, 299.0, 313.0, 315.0,
       318.0, 339.0, 348.0, 350.0, 406.0, 410.0, 411.0, 413.0, 419.0, 421.0,
       436.0, 437.0, 440.0, 450.0, 456.0, 479.0, 481.0, 490.0, 497.0, 505.0,
       513.0, 524.0, 528.0, 530.0, 543.0, 544.0, 553.0, 562.0, 569.0, 571.0,
       578.0, 580.0, 589.0, 594.0, 601.0, 626.0, 627.0, 636.0, 651.0, 674.0,
       680.0, 691.0, 693.0, 697.0, 707.0, 719.0, 753.0, 765.0, 767.0, 769.0,
       779.0, 780.0, 790.0, 802.0, 810.0, 812.0, 813.0, 836.0, 846.0, 847.0,
       855.0, 868.0, 873.0, 878.0, 882.0, 914.0],
      dtype='float64', name='book_id_mapping_x')


In [147]:
# Ensure 'Actual Rating' is correctly referenced
user_ratings = user_item_matrix_imputed  # This contains the actual ratings per user and book

In [148]:
print(user_item_with_metadata.columns)

Index([    0,     1,     3,     4,     7,     9,    11,    12,    21,    33,
       ...
       83482, 83532, 83533, 83695, 83815, 83971, 84075, 84107, 84228, 84290],
      dtype='int64', name='user_id_mapping', length=15039)


In [150]:
print(user_item_with_metadata.columns)

Index([    0,     1,     3,     4,     7,     9,    11,    12,    21,    33,
       ...
       83482, 83532, 83533, 83695, 83815, 83971, 84075, 84107, 84228, 84290],
      dtype='int64', name='user_id_mapping', length=15039)


In [149]:
# Multiply each user's rating by book metadata like num_pages and ratings_count
for col in ['num_pages', 'ratings_count'] + genre_columns:
    user_item_with_metadata[col] = user_ratings * user_item_with_metadata[col].T

# If necessary, you can fill NaNs resulting from the multiplication
user_item_with_metadata.fillna(0, inplace=True)

KeyError: 'num_pages'

In [98]:
# now create the vector for each user by combining book ratings and book metadata
# Multiply ratings by book metadata (if genre is one-hot encoded)
for col in ['num_pages', 'ratings_count']:
    user_item_with_metadata[col] = user_item_with_metadata.loc['Actual Rating'] * user_item_with_metadata[col]

# Drop book metadata columns if you want to keep just ratings, or keep them for enriched vectors
user_vectors = user_item_with_metadata.drop(['num_pages', 'ratings_count'], axis=1).T

KeyError: 'Actual Rating'

In [ ]:
#optional: dimensionality reduction - do we need this?
pca = PCA(n_components=2)  # Adjust the number of components based on your data
user_vectors_pca = pca.fit_transform(user_vectors)

In [ ]:
# and finally the clustering
kmeans = KMeans(n_clusters=3, random_state=42)  # Adjust the number of clusters
user_clusters = kmeans.fit_predict(user_vectors_pca)

# Add the cluster labels to the user vectors
user_vectors['Cluster'] = user_clusters

#check the results by printing it
print(user_vectors)